# Your First RAG Application: Stone Ridge Investor Letter Assistant

> **Note:** While the examples in this notebook use the OpenAI API, please follow the best practices outlined in the [SRHG AI Usage Guidelines](https://srhg.enterprise.slack.com/docs/T0HANKTEC/F0AB86J3A1L).

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application by building a **Stone Ridge Investor Letter Assistant**.

Imagine having an AI assistant that can answer your questions about Stone Ridge's investment philosophy, market insights, and strategic outlook based on their investor letters - that's exactly what we'll build here. We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents (Loading the Stone Ridge Investor Letter)
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation (Building the Investor Letter Assistant)
  - 🚧 Activity #1: Enhance Your Investor Letter Assistant

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utilities

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import PDFFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our PDF file into a single document in memory.

Let's look at the relevant bits of the `PDFFileLoader` class:

```python
def load_file(self):
        doc = pymupdf.open(self.path)
        text = ""
        for page in doc:
            text += page.get_text()
        self.documents.append(text)
```

We're loading the PDF document using PyMuPDF, extracting text from each page, and storing that output in our `self.documents` list.

> NOTE: We're using the Stone Ridge 2025 Investor Letter as our sample data. This content covers investment philosophy, market analysis, and strategic insights - perfect for building an investor letter assistant!

In [3]:
pdf_loader = PDFFileLoader("data/Stone Ridge 2025 Investor Letter.pdf")
documents = pdf_loader.load_documents()
len(documents)

1

In [4]:
print(documents[0][:100])

2025 Investor Letter
Investor Letter
“Every driver has a limit.  Mine is a little bit further than o


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into manageable sized chunks that retain the most relevant local context.

In [5]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

67

In [79]:
# Add a section to tag whats endnotes and whats not
new_split_docs = []
section = 'body'
for doc in split_documents:
    if 'Endnotes' in doc:
        pre, post = doc.split('Endnotes')
        if len(pre):
            new_split_docs.append((pre, 'body'))
        if len(post):
            new_split_docs.append((post, 'endnotes'))

        section = 'endnotes'
    else:
        new_split_docs.append((doc, section))
        



Let's take a look at some of the documents we've managed to split.

In [62]:
new_split_docs[0:1]

[('2025 Investor Letter\nInvestor Letter\n“Every driver has a limit.  Mine is a little bit further than others.”\n—\u2002 Ayrton Senna, greatest Formula One driver of all time\n“I’m not funny.  What I am is brave.”\n—\u2002 Lucille Ball, greatest female comedian of all time\n“I’d rather be optimistic and wrong than pessimistic and right.”\n—\u2002 Elon Musk\n“I can outlearn you.  \nI can outread you.  \nI can outthink you.  \nAnd I can outphilosophize you.”\n—\u2002 Robert De Niro as Max Cady, Cape Fear\n“I’m not superstitious, but I am a little stitious.”\n—\u2002 Michael Scott, The Office, on how Bayesians form priors \n“A wall is a very big weapon.”\n—\u2002 Banksy\nDecember 2025\nDear Fellow Investor,\nThe greatest treasure hunters know that the treasure they seek is not lost.  It is waiting.\nStarting in 1969 and for 5,954 straight days, treasure hunter Mel Fisher sought what was waiting for him: the \nwreckage of the Nuestra Señora de Atocha, a Spanish galleon that sunk in 1622. 

In [80]:
new_split_docs[-1:]

[('tion set \nforth herein. By accepting this communication, the recipient acknowledges its understanding and acceptance of the foregoing \nterms.\n',
  'endnotes')]

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [145]:
import os
import openai
from getpass import getpass

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this paper](https://arxiv.org/abs/2205.13147) for an answer to question #1.2!


##### ✅ Answer:
1. You can pass in a different value for the "dimensions" parameter to the request body
2. Open AI uses a method called Matryoshka Reprsentation Learning where the embeddings created are increasing in granularity. The idea being if you took the first N dimensions, that should be comparable to embedding in N dimensions, each incremental dimension adds to the granularity of the embedding but does not reflect a change to its previous embeddings. This is sort of like a decimal, where you can truncate the last n digits without losing any of the information in the first n digits. By including more digits, or dimensions, you only get to a more granular representation of the information. 

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [81]:
from typing import *
from sentence_transformers import SentenceTransformer

from aimakerspace.vectordatabase import *


class EmbeddingModel:
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(list_of_text)
        return list(embeddings)

    async def async_get_embedding(self, text: str) -> List[float]:
        return self.model.encode(text)

    def get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(list_of_text)
        return list(embeddings)

    def get_embedding(self, text: str) -> List[float]:
        return self.model.encode(text)


class VectorDatabase:
    def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()

    def insert(self, key: str, vector: np.array) -> None:
        self.vectors[key] = vector

    def search(
        self,
        query_vector: np.array,
        k: int,
        distance_measure: Callable = cosine_similarity,
    ) -> List[Tuple[str, float]]:
        scores = [
            (key, distance_measure(query_vector, vector))
            for key, vector in self.vectors.items()
        ]
        return sorted(scores, key=lambda x: x[1], reverse=True)[:k]

    def search_by_text(
        self,
        query_text: str,
        k: int,
        distance_measure: Callable = cosine_similarity,
        return_as_text: bool = False,
    ) -> List[Tuple[str, float]]:
        query_vector = self.embedding_model.get_embedding(query_text)
        results = self.search(query_vector, k, distance_measure)
        return [result[0] for result in results] if return_as_text else results

    def retrieve_from_key(self, key: str) -> np.array:
        return self.vectors.get(key, None)

    async def abuild_from_list(self, list_of_text: List[Tuple[str, str]]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings([t[0] for t in list_of_text])
        for (text, metadata), embedding in zip(list_of_text, embeddings):
            if metadata == 'body':
                self.insert(text, np.array(embedding))
        return self

em = EmbeddingModel()


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [82]:
vector_db = VectorDatabase(em)
vector_db = asyncio.run(vector_db.abuild_from_list(new_split_docs))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
Since this is presumably IO bound, using async means you can send all your requests in one shot (or a few batches) and wait for the responses, this allows us to lean on any horizontal scaling in the openai embedding api. 

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [84]:
vector_db.search_by_text("What is Stone Ridge's investment philosophy?", k=3)

[('hen we believe \nit is time – have been the hallmarks of our operating model since firm inception. \nThe first rule of product design at Stone Ridge is we build products we want for ourselves.  Stone Ridge is always \nthe seed investor and, whenever possible – but not yet always – the largest.  Consider our “no-device” policy in \nthis context.  Would you ever pull out your phone and start scrolling in the middle of a meeting with your largest \ninvestor?  At Stone Ridge, we do not pull our phones out on each other.\nWith Stone Ridge increasingly Stone Ridge’s largest investor, the scale of our profits as principals, not agents, creates \na level of alignment with our outside investors we are unaware exists elsewhere.  Those investors occasionally \nremark that this is a wonderfully clarifying and palpable distinction, and that Stone Ridge feels different inside. \nEarlier this year, Stone Ridge Holdings Group (SRHG), parent company of Stone Ridge Asset Management, NYDIG, \nand Long

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicity's sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analogous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
You can use the instructions api to pass in instructions that you may otherwise put in the developer role message. I guess this is theoretically useful if you want to keep a conversation going and only want to apply instructions to certain messages, i wonder if you removed the developer message on those conversation lines if it would confuse the llm. Not clear how useful this is?


### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [92]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [93]:
print(response)

Hello! The "best" way to write a loop in Python depends on what you're trying to achieve, but generally, Python's `for` loops and `while` loops are both very useful and have their ideal use cases.

Here’s a brief overview to help you write effective loops:

### Using a `for` loop
`for` loops are great when you want to iterate over a sequence (like a list, tuple, string, or range).

Example:
```python
# Looping through a list
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(fruit)
```

If you want to loop a specific number of times:
```python
for i in range(5):
    print(i)
```

### Using a `while` loop
`while` loops are useful when you want to repeat something until a condition changes.

Example:
```python
count = 0
while count < 5:
    print(count)
    count += 1
```

### Best Practices
- Prefer `for` loops when iterating over collections or a known number of iterations.
- Use `while` loops when the number of repetitions depends on a condition changing within the 

## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [94]:
RAG_SYSTEM_TEMPLATE = """You are a helpful investor letter assistant that answers questions about Stone Ridge's investment philosophy, market insights, and strategic outlook based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't have information about that in the investor letter"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Include a reminder that this is for informational purposes only and not investment advice when appropriate
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [95]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [96]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What are the key investment themes discussed in the letter?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: Based on the provided excerpts from the investor letter, several key investment themes emerge:

1. **Pursuit of Competitive Advantage Through Sophistication and Operational Excellence**  
   The letter discusses Stone Ridge's initial belief that breakthroughs in proprietary financing and hedging technology, resulting in an edge in cost of funds, would suffice for best-in-class investment performance in energy markets (Source 1). However, they learned that financial sophistication alone is insufficient without industrial control and operational expertise. This led them to partner with Flywheel Energy, recognized for superior operations partly due to their leadership's elite military backgrounds. This highlights a theme of combining financial innovation with operational mastery to achieve superior investment outcomes.

2. **Long-Term Perseverance and Search for Value**  
   Stone Ridge draws an analogy to the long search for the treasure of the Atocha shipwreck, emphasizing pat

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
You could jack up the reasoning effort, the other thing would be using the developer role, but its not clear to me how this is different than the system role - they seem to be somwhat interchangeable.


### 🏗️ Activity #1:

Enhance your Stone Ridge Investor Letter Assistant in some way! 

Suggestions are: 

- **Multi-Document Support**: Allow it to ingest multiple investor letters from different years for historical comparison
- **New Distance Metric**: Implement a different similarity measure - does it improve retrieval for financial content?
- **Metadata Support**: Add metadata like year, topic categories (market outlook, strategy, performance) to help filter results
- **Different Embedding Model**: Try a different embedding model - does domain-specific tuning help for financial content?
- **Multi-Source Ingestion**: Add the capability to ingest content from SEC filings, earnings calls, or other financial documents

While these are suggestions, you should feel free to make whatever augmentations you desire! Think about what features would make this investor letter assistant most useful for understanding Stone Ridge's investment approach.

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [125]:
import os
import re

docs_by_year = {}
for fname in os.listdir('./data'):
    year = str(int(re.findall(r'\d+', fname)[0]))
    print(os.path.join('./data', fname), year)
    pdf_loader = PDFFileLoader(os.path.join('./data', fname))
    documents = pdf_loader.load_documents()
    assert len(documents) == 1
    
    assert 'Endnotes' in documents[0]
    relevant_doc = documents[0].split('Endnotes')[0]
    text_splitter = CharacterTextSplitter()
    split_documents = text_splitter.split_texts([relevant_doc])
    docs_by_year[year] = split_documents
    print(year, len(split_documents))


./data/2023 Stone Ridge Investor Letter No Front Image.pdf 2023
2023 60
./data/Stone_Ridge_2024_Investor_Letter.pdf 2024
2024 86
./data/2020 Stone Ridge Investor Letter No Front Image.pdf 2020
2020 75
./data/Stone Ridge 2025 Investor Letter.pdf 2025
2025 56


In [128]:
from typing import *
from sentence_transformers import SentenceTransformer

from aimakerspace.vectordatabase import *


class EmbeddingModel:
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(list_of_text)
        return list(embeddings)

    async def async_get_embedding(self, text: str) -> List[float]:
        return self.model.encode(text)

    def get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(list_of_text)
        return list(embeddings)

    def get_embedding(self, text: str) -> List[float]:
        return self.model.encode(text)


class VectorDatabase:
    def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors_by_year = defaultdict(lambda: defaultdict(np.array))
        self.embedding_model = embedding_model or EmbeddingModel()

    def insert(self, year: str, key: str, vector: np.array) -> None:
        self.vectors_by_year[year][key] = vector

    def search(
        self,
        query_vector: np.array,
        k: int,
        distance_measure: Callable = cosine_similarity,
        year = None
    ) -> List[Tuple[str, float]]:
        scores = [
            (year, key, distance_measure(query_vector, vector))
            for year, key_vector in self.vectors_by_year.items() for key, vector in key_vector.items()
        ]
        if year is not None:
            scores = [a for a in scores if a[0] == year]
        return sorted(scores, key=lambda x: x[2], reverse=True)[:k]

    def search_by_text(
        self,
        query_text: str,
        k: int,
        distance_measure: Callable = cosine_similarity,
        return_as_text: bool = False,
        year = None,
    ) -> List[Tuple[str, float]]:
        query_vector = self.embedding_model.get_embedding(query_text)
        results = self.search(query_vector, k, distance_measure, year=year)
        return [result[1] for result in results] if return_as_text else results

    def retrieve_from_key(self, year, key: str) -> np.array:
        return self.vectors.get(year, {}).get(key, None)

    async def abuild_from_list(self, list_of_text_by_year: Dict[str, List[str]]) -> "VectorDatabase":
        for year, list_of_text in list_of_text_by_year.items():
            embeddings = await self.embedding_model.async_get_embeddings([t[0] for t in list_of_text])
            for text, embedding in zip(list_of_text, embeddings):
                self.insert(year, text, np.array(embedding))
        return self

em = EmbeddingModel()
vector_db = VectorDatabase(em)
vector_db = asyncio.run(vector_db.abuild_from_list(docs_by_year))

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [157]:
ChatOpenAI().run(['hi'])

InternalServerError: processing failed: failed to parse request: invalid type: string "hi", expected struct RequestMessage at line 1 column 17

In [172]:
from pydantic import BaseModel
from openai import OpenAI
client = OpenAI()
class QueryResponseGetter(BaseModel):
    years_to_query: List[str]
    query: str

# response = client.responses.parse(
#     model='gpt-4.1-mini',
#     input=[
#         {
#             'role': 'developer',
#             'content': f'I have a vector db of stoneridge investor letters from the following years {", ".join(vector_db.vectors_by_year.keys())} based on teh following user question, what years should i query and what should i query with'
#         },
#         {
#             'role': 'user',
#             'content': 'what were the key highlights of the 2020 investor letter'
#         }
#     ],
#     text_format=QueryResponseGetter
# )
model='gpt-4.1-mini'
# model='gpt-5'
context = [
  { "role": "system", "content": "You are a helpful assistant." },
  { "role": "user", "content": "Hello!" }
]


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

response = client.responses.create(
    model=model,
    input=[
        {
            'role': 'developer',
            'content': f'I have a vector db of stoneridge investor letters from the following years {", ".join(vector_db.vectors_by_year.keys())} based on the following user question, what years should i query and what should i query with'
        },
        {
            'role': 'user',
            'content': 'what were the key highlights of the 2020 investor letter'
        }
    ],
    # output_config="format": {
    #         "type": "json_schema",
    #         "schema": {
    #             "type": "object",
    #             "properties": {
    #                 "name": {"type": "string"},
    #                 "email": {"type": "string"},
    #                 "plan_interest": {"type": "string"},
    #                 "demo_requested": {"type": "boolean"}
    #             },
    #             "required": ["name", "email", "plan_interest", "demo_requested"],
    #             "additionalProperties": False
    #         }
    #     }
)

# event = response.output_parsed

TypeError: Responses.create() got an unexpected keyword argument 'messages'

In [167]:
completion

ChatCompletion(id='chatcmpl-D5hVsdehwrMt0EWzNSQwTKJlrygGr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Great question! In Python, “best” depends on what you’re trying to do. Here’s a quick, Pythonic guide with patterns to choose from:\n\n- Iterate over items (default)\n  for x in items:\n      ...\n\n- Need index and value\n  for i, x in enumerate(items):  # optionally enumerate(items, start=1)\n      ...\n\n- Iterate two (or more) sequences together\n  for a, b in zip(a_list, b_list):\n      ...\n\n- Fixed number of times\n  for _ in range(n):\n      ...\n\n- Loop until a condition changes\n  while condition:\n      ...\n\n- Build a new collection (prefer comprehensions)\n  squares = [x*x for x in nums]              # list\n  evens = {x for x in nums if x % 2 == 0}    # set\n  mapping = {x: x*x for x in nums}           # dict\n\n- Stream values without making a list (generator)\n  total = sum(x*x for x in nums if x > 0)\n\n- Se

In [ ]:
response

In [132]:
response

dict_keys(['2023', '2024', '2020', '2025'])

In [135]:
from openai import OpenAI
client = OpenAI()

In [158]:
from aimakerspace.openai_utils.chatmodel import ChatOpenAI
chat_openai = ChatOpenAI()

In [160]:
chat_openai.run([{'role': 'user', 'content': 'say hello'}])

'Hello! How can I assist you today?'

In [129]:
vector_db.search_by_text('what is the best part of waking up', 4)

[('2023',
  'use of this year. Success, and failure, are lagging indicators. Mountain \nclimbing disasters are always a series of small, seemingly inconsequential decisions that interact in unexpected \nways, compounding exponentially. Success works the same way. \nIt is fashionable these days to say investment outcomes follow a Power Law. That’s technically true and soulless. At \nStone Ridge, the compounding we’re after is in wisdom and trust in our relationships with each other. Life is much \nmore satisfying when we realize that relationships also follow a Power Law, and invest accordingly.\nSecond, we let the simplicity of our business model be enough. The humility of our questions and the plainness of our \nactions – our undefended openness – may occasionally seem to border on irresponsible naivete, but we rarely cross \nthat border. And even when we do, we have each other’s backs with such ferocity that we never fail to pull ourselves \nback to safety. \nOur most sustainable com